In [3]:
import numpy as np
import sys
import os

# Absolute path to the folder that contains "quant_rotor"
sys.path.append("/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems")

#working with kietic and potential operators

from quant_rotor.models.dense.support_ham import write_matrix_elements, basis_m_to_p_matrix_conversion

from quant_rotor.models.sparse.support_ham import build_V_in_p

#ED hamiltonian implementation.
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

# Iterative residual calculation
from quant_rotor.core.dense.t_amplitudes_guess import amplitute_energy, t_1_amplitude_guess_ground_state, t_2_amplitude_guess_ground_state
from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic
from quant_rotor.core.dense.t_amplitudes_non_periodic import t_non_periodic

from quant_rotor.core.dense.t_amplitudes_periodic_fast import t_periodic # for bigger systems (much harder to modify)

#Time dependant methods
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme

from quant_rotor.core.dense.de_solve_one_thermal_dense import integration_scheme as integration_scheme_fast # for bigger systems (much harder to modify)

# Producing hamiltonian operators.

## Producing Kinetic and Potential operators.

In [2]:
state = 3
g = 0.1
tau = 0

Example of the call to produce kinetic and potential matricies.

In [15]:
# Recomended to produce small matrix which can be printed and see how it changes. For example 3 state system

Kinetic, Potential = write_matrix_elements((state-1)//2, tau) # takes input of the number of unique states see doc-string and annotation for more information

K, V = Kinetic, Potential

V_tensor = V.reshape(state, state, state, state)  # Adjust if needed

h_full = basis_m_to_p_matrix_conversion(K, state)
v_full = basis_m_to_p_matrix_conversion(V_tensor, state)

In [16]:
h_full

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [17]:
v_full.reshape(state**2, state**2)

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  , -0.25,  0.75],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

In [18]:
Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

# Example of how to convert to dense.

Kinetic_dense = Kinetic_sparse.toarray()
Potential_dense = Potential_sparse.toarray()

In [19]:
Kinetic_dense

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [20]:
Potential_dense

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  , -0.25,  0.75],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

Check the output.

In [21]:
print(np.array_equal(h_full, Kinetic_dense))
print(np.allclose(v_full.reshape(state**2, state**2), Potential_dense), 1e-30)

True
True 1e-30


Produce several matricies for tests.

In [26]:
save_dir = "example_data/kinetic_potential_matricies"

for state in range(3, 15, 2):
    
    Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    save_path = os.path.join(save_dir, f"kin_pot_state_{state}.npz")
    np.savez(save_path, K=Kinetic_dense, V=Potential_dense)

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


In [28]:
save_dir = "example_data/kinetic_potential_matricies"

for state in range(3, 15, 2):

    Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    save_path = os.path.join(save_dir, f"kin_pot_state_{state}.npz")

    data = np.load(save_path)

    print(np.array_equal(Kinetic_dense, data["K"]))
    print(np.array_equal(Potential_dense, data["V"]))

True
True
True
True
True
True
True
True
True
True
True
True


## Producing hamiltonians.

In [4]:
site = 5
state = 3
g = 0.1

Exact Diognalisation hamiltonian.

In [5]:
Hamiltonian, K, V = hamiltonian_dense(state, site, g)

H = Hamiltonian

eig_val_ED, eig_vec_ED = np.linalg.eigh(H) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

Natural Orbitals hamiltonian.

In [6]:
H, K, V = hamiltonian_general_dense(state, site, g)

eig_val_NO, eig_vec_NO = np.linalg.eigh(H) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

A way to check the NO with ED we can check the first #states biggest energyes and eigenstates.

In [7]:
index_ED = np.argsort(-eig_val_ED)
index_NO = np.argsort(-eig_val_NO)

# Makes a change of basis matrix.
np.array_equal(eig_vec_ED[:, index_ED[:state]], eig_vec_NO[:, index_NO[:state]])

False

In [8]:
eig_val_ED[index_ED] - eig_val_NO[index_NO]

array([ 0.019153222837,  0.005319058582,  0.001430636417,  0.019003612539,  0.014427689401,  0.01538692423 ,  0.005131016239,  0.006576208265,  0.0058758929  ,  0.010833602921,  0.015074671301,  0.010183114955,  0.000528628982,  0.002820779268,  0.003288094698,  0.00488808825 ,  0.002579507218,  0.00260220316 ,  0.003177680065,  0.016283574132,  0.016155445754,  0.002521521823,  0.002740856843,  0.002061699768,  0.00071498671 ,  0.000867474444,  0.002925281218,  0.002925281218,  0.002902362984,  0.002902362984, -0.            ,  0.            , -0.013517845464,  0.028445053748,  0.017470423121,  0.068323095365,  0.046851398671,  0.046851398671,  0.030683844065,  0.021613800348,  0.068332156841,  0.068332156841,  0.055816102944,  0.057094599779,  0.037402447625,  0.045777306562,  0.029238545148,  0.033828401261,  0.036034367092,  0.036034367092,  0.033648472344,  0.033113681644,  0.045111826005,  0.020803476335,  0.02448084637 , -0.00184118531 ,  0.027928080435,  0.008173260541,  0.0091